In [3]:
import numpy as np 
from math import log, e 
from scipy.stats import norm 
from calcbsimpvol import calcbsimpvol

class BSM: 
    def __init__(self, args): 
        self.mkt_prc = float(args[0]) # Option Market Price 
        self.Type = int(args[1]) # Option Type 
        self.S = float(args[2]) # Underlying Price 
        self.K = float(args[3]) # Strike price 
        self.r = float(args[4]) # Interest Rate 
        self.q = float(args[5]) # Dividend Rate 
        self.T = float(args[6]) / 365.0 # Time to Maturity 

        self.sigma = self.imp_vol() # Implied volatility 
        self.sigmaT = self.sigma * self.T ** 0.5 
        self.d1 = (log(self.S / self.K) + (self.r - self.q + 0.5 * (self.sigma ** 2)) * self.T) / self.sigmaT  
        self.d2 = self.d1 - self.sigmaT 
        
    def imp_vol(self): 
        cp = np.asarray(self.Type) 
        P = np.asarray([self.mkt_prc]) 
        S = np.asarray(self.S) 
        K = np.asarray([self.K]) 
        tau = np.asarray([self.T]) 
        r = np.asarray(self.r) 
        q = np.asarray(self.q) 
        imvol = calcbsimpvol(dict(cp=cp, P=P, S=S, K=K, tau=tau, r=r, q=q)) 
        imvol = np.ndarray.item(imvol) 
            
        return imvol 

    def delta(self): 
        dfq = e ** (-self.q * self.T) 
            
        if self.Type == 1: 
            return dfq * norm.cdf(self.d1) 
        else: 
            return dfq * (norm.cdf(self.d1) - 1) 

    def gamma(self): 
        return e ** (-self.q * self.T) * norm.pdf(self.d1) / (self.S * self.sigmaT) 
        
    def vega(self): 
        return 0.01 * self.S * e ** (-self.q * self.T) * norm.pdf(self.d1) * self.T ** 0.5 
        
    def theta(self): 
        df = e ** -(self.r * self.T) 
        dfq = e ** (-self.q * self.T) 
        tmptheta = (1.0 / 365.0) * (-0.5 * self.S * dfq * norm.pdf(self.d1) * self.sigma / (self.T ** 0.5) + self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) - self.r * self.K * df * norm.cdf(self.Type * self.d2))) 
        return tmptheta 
        
    def rho(self): 
        df = e ** -(self.r * self.T) 
            
        return self.Type * self.K * self.T * df * 0.01 * norm.cdf(self.Type * self.d2)

In [4]:
mkt_prc = 0.31 # Market Price 
option_type = 1 # Option Type 
S = 268.55 # Underlying Price 
K = 275.0 # Strike Price 
q = 0.0 # Dividend Rate 
r = 0.01 # Interest Rate 
T = 9 # Time to Maturity 

args = [mkt_prc, option_type, S, K, r, q, T] 

BSM = BSM(args) 
imp_vol = BSM.imp_vol() 
delta = BSM.delta() 
gamma = BSM.gamma() 
vega = BSM.vega() 
theta = BSM.theta() 
rho = BSM.rho() 

print('Implied Vol = ', round(imp_vol, 4)) 
print('Delta = ', round(delta, 4)) 
print('Gamma = ', round(gamma, 4)) 
print('Vega = ', round(vega, 4)) 
print('Theta = ', round(theta, 4)) 
print('Rho = ', round(rho, 4))

Implied Vol =  0.126
Delta =  0.1196
Gamma =  0.0376
Vega =  0.0842
Theta =  -0.0598
Rho =  0.0078
